Just for developing the CryoEnv2

In [9]:
import numpy as np 
import matplotlib.pyplot as plt
import cait as ai
import cait.versatile as vai

# Get multiprocessing
n_processes = ai._available_workers
# Standard values for plotting
backend = "plotly" # Choose "mpl" for keeping plots when closing, "plotly" for interactive plotting

# Go up one level because notebook is in folder
import sys
import os
sys.path.append(os.path.abspath(".."))  # Go to project root folder

# New cryoEnv implementation
from CryoEnv2 import CryoEnv as cryo

In [10]:
# Create datahandler to load with generated testpulses

# File directory
fpath = "../Files/"
fname = "test_file"

# Set time
fs = 50e3   # We use 50 kHz from VDAQ

# Create datahandler
dh = ai.DataHandler(channels=["Heater", "Output"], sample_frequency=fs)
dh.set_filepath(path_h5=fpath, fname=fname, appendix=False)
dh.init_empty()
dh

DataHandler Instance created.


DataHandler(record_length=16384, sample_frequency=50000.0, dt_us=20, channels=['Heater', 'Output'])

In [11]:
# Should be empty before adding content
dh.content()

Heater constant
  event                       (1, 10, 16384)  float32
  time_mus                    (10,)           float32
  time_s                      (10,)           float32


# Creating DAC group
The DAC is the voltage value set to the Heater.
This is usually a constant DC, which is superimposed with Testpulses.
The constant part sets the OP of the detector.

All the simulation of the Heater, testpulses etc should be handled by the `CryoEnv2`, while the data itself will be stored in the datahandler from cait.

In [12]:
# This function makes writing to dh easier
def event_to_dh(dh, channel, events, time=None):
    if time is not None:
        raise KeyError("Time information is not yet implemented")
    events = np.expand_dims(events, axis=0)
    dh.set(channel, event=events)
    
    # Generate random time scale, since this is not really implemented yet
    time = np.linspace(0, 10, events.shape[1])  
    dh.set(channel, time_s = time)
    dh.set(channel, time_mus = time*1e3)

In [13]:
# Init an cryo environment
test_cryo_env = cryo(dh.record_length)

# Before we can use the heater, we have to initialize it
test_cryo_env.init_heater(resistance=10e-3)  # Typical resistance value: 10 mOh

# Now we create 10 constant traces with each a voltage of 0.5
Constant_traces = test_cryo_env.get_constant_heater_trace(voltage=0.5, number=10)  # Set constant heater value to 0.5 V

# Write to datahandler
event_to_dh(dh=dh, channel="Heater constant", events=Constant_traces)


Heater initialized.


/opt/anaconda3/envs/CAITdev/lib/python3.12/site-packages/cait/data_handler.py:1021: UserWarning:

Dataset event already exists in group Heater constant. To change it, set 'change_existing' or 'overwrite_existing' to True


/opt/anaconda3/envs/CAITdev/lib/python3.12/site-packages/cait/data_handler.py:1021: UserWarning:

Dataset time_s already exists in group Heater constant. To change it, set 'change_existing' or 'overwrite_existing' to True


/opt/anaconda3/envs/CAITdev/lib/python3.12/site-packages/cait/data_handler.py:1021: UserWarning:

Dataset time_mus already exists in group Heater constant. To change it, set 'change_existing' or 'overwrite_existing' to True




In [14]:
dh.content()

Heater constant
  event                       (1, 10, 16384)  float32
  time_mus                    (10,)           float32
  time_s                      (10,)           float32


In [15]:
# Now we can see through this with the cait.versatile event_iterator!
vai.Preview(dh.get_event_iterator("Heater constant"), backend=backend)

Nice :) We can use constant Heater values already